# Generation of the lines for the A-OCR Entomology collection

In [22]:
dir_images = "/home/ialzuru/Spring2019/H-MaTE/biocolls/aocr_entomology/images"
dir_output = "/home/ialzuru/Spring2019/H-MaTE/biocolls/aocr_entomology/lines_creation_gcp"

## 1. Removal of the bar codes and rulers
Bar codes and rulers insert many segmentation errors. This process eliminates most of them, creating new "clean" images.<br>
This is a multicore process which minimizes the quality lost in the images.

<font color=blue>time python3 ../ALOT/**delBarRuler_mt.py** -i /home/ialzuru/Spring2019/H-MaTE/biocolls/aocr_entomology/images -o /home/ialzuru/Spring2019/H-MaTE/biocolls/aocr_entomology/lines_processing/cleaned_images</font><br>
real	0m13.156s<br>
<br>
NOTE: The algorithm did not generate the expected results (some areas of the rulers are still present). Therefore, I decided to proceed with the original images.

## 2. Size adjustment of the images to comply with the Google Cloud 10 MB's maximum

<font color=blue>python3 ../ALOT/**resizeDir_mt.py** -i dir_images -o dir_output + "/" + resized_images</font>
<br>
This step was not neccesary for A-OCR Entomology, where the biggest image occupies 1.2 MB.

## 3. Segmentation of the images in lines, using the Google Cloud OCR.
The Google Cloud OCR does not work with lines, like OCRopus or Tesseract. But, taking the coordinates of the symbols' bounding boxes, I was able to generate the lines from the information provided by the GCP OCR<br>
This step must be executed from a Google Cloud project and is not free. It has a cost per processed image.<br><br>
Google credentials need to be enabled:<br>
<font color=blue>source /home/ialzuru/Terms/Fall2018/Google/env/bin/activate<br>
export GOOGLE_APPLICATION_CREDENTIALS="/home/ialzuru/Terms/Fall2018/Google/hardy-album-221622-fb268d75b31e.json"</font>

The command to extract all the lines of the collection is the following:<br>
<font color=blue>
time for i in ~/Spring2019/H-MaTE/biocolls/aocr_entomology/images/&ast;.jpg; do<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;b=\`basename \\$i .jpg\`;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;python3 ~/Spring2019/ALOT/**get_lines_google.py** -if \\$i -od ~/Spring2019/H-MaTE/biocolls/aocr_entomology/lines_creation_gcp/\\$b -of ~/Spring2019/H-MaTE/biocolls/aocr_entomology/lines_creation_gcp/lines.txt;<br>
done
</font>
<br><br>
real	1m49.930s

## 4. Arrangement of the lines and directories.
Creation of the directories and copy of the files in separated folders.<br>
<font color=blue>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/fulltext_ocred<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_bin<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_txt<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_prob<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_ocropus_binarization<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/fulltext_ocred<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/lines_txt<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/lines_prob<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/fulltext_ocred<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_img<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_txt<br>
    mkdir ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_prob<br>
    <br>
    for i in ~/Spring2019/H-MaTE/biocolls/aocr_entomology/lines_processing/google/&ast;; do<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cp \\$i/&ast;.jpg ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_img/<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cp \\$i/&ast;.txt ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_txt/<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cp \\$i/&ast;.prob ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_prob/<br>
    done<br>
    <br>
    cd ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_txt<br>
    find . ! -iregex ".&ast;_[0-9][0-9][0-9]\\.txt" -exec mv {} ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/fulltext_ocred/ \\;
</font>

In [23]:
n_lines = !ls ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_img | wc -l
n_lines = int(n_lines[0])
print("Lines in the collection: " + str(n_lines))

Lines in the collection: 1132


## 5. Binarization of the lines using OCRopus
OCRopus recognizer works better with binary images.

<font color=blue>time python2 ~/Spring2019/ALOT/**binarizeDir_mt.py** -if ~/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_img -of ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_ocropus_binarization</font><br>
real	1m46.406s

The binarized files are moved to the <font color=blue>lines_bin</font> directory and renamed with their correspondent image name.<br>
<font color=blue>
time for i in ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_ocropus_binarization/&ast;; do<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;b=\`basename \\$i\`;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cp \\$i/0001.bin.png ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_bin/\\$b.bin.png;<br>
done<br>
</font>
real	0m4.933s

## 6. OCRopus Recognition: Generation of the text and probability files
Execution of OCRopus on the binarized lines.

<font color=blue>time python2 ~/Spring2019/ALOT/**recognizeDir_mt.py** -if ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_bin -mf ~/ocropus/ocropy/models -mn en-default.pyrnn.gz -p True</font>
<br>
real	12m36.581s  (6 cores utilized)<br>
<br>
The text and probability files are moved from the default location: <font color=blue>lines_bin</font>, to independent folders: <font color=blue>lines_txt</font> and <font color=blue>lines_prob</font>.<br>
<br>
<font color=blue>
    mv ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_bin/&ast;.txt ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_txt/<br>
    mv ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_bin/&ast;.prob ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_prob/
</font>

In [24]:
n_lines_ocropus = !ls ~/Spring2019/H-MaTE/biocolls/aocr_entomology/ocropus/lines_txt | wc -l
n_lines_ocropus = int(n_lines_ocropus[0])
print("OCR-ed number of lines with OCRopus: " + str(n_lines_ocropus))

OCR-ed number of lines with OCRopus: 1132


## 7. Tesseract Recognition: Generation of the text and probability files
Execution of Tesseract on the colored lines generated through the GCP OCR.

<font color=blue>time python ~/Spring2019/ALOT/**tessDir_mt.py** -id /home/ialzuru/Spring2019/H-MaTE/biocolls/aocr_entomology/google/lines_img -td ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/lines_txt -cd ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/lines_prob</font><br>
real	48m4.321s

In [25]:
n_lines_tesseract = !ls ~/Spring2019/H-MaTE/biocolls/aocr_entomology/tesseract/lines_txt | wc -l
n_lines_tesseract = int(n_lines_tesseract[0])
print("OCR-ed number of lines with OCRopus: " + str(n_lines_tesseract))

OCR-ed number of lines with OCRopus: 949
